In [1]:
import pandas as pd
import numpy as np

c:\Users\Pc\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [66]:
# Kolon isimlerini belirleyerek dosyayı okuma
column_names = ['label', 'message']  # Kolon isimleri
df = pd.read_csv(r'C:\Users\Pc\Desktop\SMSSpamCollection.tsv', sep='\t', header=None, names=column_names, encoding='utf-8')

In [97]:
# Label'ları sayısal değerlere dönüştürme
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

In [135]:
df

,label,message
0,0,I've been searching for the right words to tha...
1,1,Free entry in 2 a wkly comp to win FA Cup fina...
2,0,"Nah I don't think he goes to usf, he lives aro..."
3,0,Even my brother is not like to speak with me. ...
4,0,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5563,1,This is the 2nd time we have tried 2 contact u...
5564,0,Will ü b going to esplanade fr home?
5565,0,"Pity, * was in mood for that. So...any other s..."
5566,0,The guy did some bitching but I acted like i'd...


In [8]:
df['kelime'] = df['message'].apply(lambda x : x.split())

In [14]:
max_kelime = []

for i in range(0,df.shape[0]):
    max_kelime.append(len(df['kelime'][i]))
    if len(df['kelime'][i]) == 171:
        print(i)
print(max(max_kelime))

1081
171


## 1. DataSet, Train-Test-Validation Split

In [136]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, stratify=df['label'])
test, validation = train_test_split(test, test_size=1/3, stratify=test['label'])

train.shape, test.shape, validation.shape

((3897, 2), (1114, 2), (557, 2))

In [137]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict(
    {'train':Dataset.from_pandas(train, preserve_index=False),       # indeksleri almıyor çünkü indeksler rastgele alındığı için farklı
     'test':Dataset.from_pandas(test, preserve_index=False),
     'validation': Dataset.from_pandas(validation, preserve_index=False)
     }  
)

dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'message'],
        num_rows: 3897
    })
    test: Dataset({
        features: ['label', 'message'],
        num_rows: 1114
    })
    validation: Dataset({
        features: ['label', 'message'],
        num_rows: 557
    })
})

- DatasetDict ve Dataset kullanmak, modelinizin eğitim sürecini daha verimli ve düzenli hale getirir.
- Eğer BERT veya başka bir model kullanıyorsanız, bu yapıları kullanmak, veri setlerinizi yönetmek ve modelinizi eğitmek için en iyi uygulamadır. 

## 2. Tokenization

In [138]:
from transformers import AutoTokenizer

model_ckpt = "mshenoda/roberta-spam"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [139]:
def tokenize(batch):
    return tokenizer(batch['message'], padding=True, truncation=True)

In [ ]:
emotion_encoded = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/3897 [00:00<?, ? examples/s]

Map:   0%|          | 0/1114 [00:00<?, ? examples/s]

Map:   0%|          | 0/557 [00:00<?, ? examples/s]

In [115]:
# label2id, id2label
label2id = {'ham' : 0, 'spam' : 1}
id2label = {0 : 'ham',  1 : 'spam'}

label2id, id2label

({'ham': 0, 'spam': 1}, {0: 'ham', 1: 'spam'})

In [116]:
emotion_encoded['train'][0].items()

dict_items([('label', 'ham'), ('message', "I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times."), ('input_ids', [0, 100, 348, 57, 6062, 13, 5, 235, 1617, 7, 3392, 47, 13, 42, 31997, 1843, 4, 38, 4198, 939, 40067, 185, 110, 244, 13, 4159, 8, 40, 19805, 127, 4198, 4, 370, 33, 57, 4613, 8, 10, 14164, 23, 70, 498, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 3. Model Building

In [117]:
from transformers import AutoModel 
import torch

model = AutoModel.from_pretrained(model_ckpt)

Some weights of RobertaModel were not initialized from the model checkpoint at mshenoda/roberta-spam and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- model = AutoModel.from_pretrained(model_ckpt) satırı gereksizdir eğer AutoModelForSequenceClassification kullanıyorsanız. İşte nedenleri:

- Model Seçimi: AutoModel sınıfı, modelin genel bir versiyonunu yükler, yani sınıflandırma başlığı (classifier head) olmayan bir modeldir. Eğer amacınız bir sınıflandırma (classification) görevi gerçekleştirmekse, bu yeterli olmaz.

- Sınıflandırma İçin Uygun Model: AutoModelForSequenceClassification sınıfı, belirli bir sınıflandırma görevi için gerekli olan son katmanı (classifier head) içeren bir model döndürür. Bu, genellikle metin sınıflandırma gibi görevlerde kullanılır.

In [132]:
model.config

RobertaConfig {
  "_name_or_path": "mshenoda/roberta-spam",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.45.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

## 4. Fine-Tuning Transformers

In [100]:
from transformers import AutoModelForSequenceClassification, AutoConfig

num_labels = len(label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

In [146]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

In [142]:
from transformers import TrainingArguments

batch_size = 64
training_dir = r"C:\Temp\bert_base_train_dir"

training_args = TrainingArguments(output_dir=training_dir,
                                  overwrite_output_dir = True,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch',
                                  disable_tqdm = False
)

c:\Users\Pc\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## 5. Build Model and Trainer

In [119]:
# use sklearn to build compute metrics
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}

In [147]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset = emotion_encoded['train'],
                  eval_dataset = emotion_encoded['validation'],
                  tokenizer = tokenizer)

In [ ]:
trainer.train() #  pc cöktü      #distilbert halletti sriden

  0%|          | 0/122 [00:00<?, ?it/s]

In [2]:
df = pd.read_excel("https://github.com/laxmimerit/All-CSV-ML-Data-Files-Download/raw/master/fake_news.xlsx")

In [3]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [20]:
df.dropna(inplace = True)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [98]:
from sklearn.model_selection import train_test_split

# 70% for training, 20% test, 10% validation
train, test = train_test_split(df, test_size=0.3, stratify=df['label'])
test, validation = train_test_split(test, test_size=1/3, stratify=test['label'])

train.shape, test.shape, validation.shape, df.shape

((3897, 2), (1114, 2), (557, 2), (5568, 2))

In [99]:
from datasets import Dataset, DatasetDict

dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train, preserve_index=False),
        "test": Dataset.from_pandas(test, preserve_index=False),
        "validation": Dataset.from_pandas(validation, preserve_index=False)
    }
)
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'message'],
        num_rows: 3897
    })
    test: Dataset({
        features: ['label', 'message'],
        num_rows: 1114
    })
    validation: Dataset({
        features: ['label', 'message'],
        num_rows: 557
    })
})

In [100]:
from transformers import AutoTokenizer

text = "Machine learning is awesome!! Thanks KGP Talkie."

model_ckpt = "distilbert-base-uncased"
distilbert_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
distilbert_tokens = distilbert_tokenizer.tokenize(text)

model_ckpt = "google/mobilebert-uncased"
mobilebert_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
mobilebert_tokens = mobilebert_tokenizer.tokenize(text)

model_ckpt = "huawei-noah/TinyBERT_General_4L_312D"
tinybert_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
tinybert_tokens = tinybert_tokenizer.tokenize(text)

In [101]:
distilbert_tokenizer, mobilebert_tokenizer, tinybert_tokenizer

(DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
 	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 MobileBertTokenizerFast(name_or_path='google/mobilebert-uncased', vocab_size=30522, model_max_length=1000

In [102]:
def tokenize(batch):
    return distilbert_tokenizer(batch['message'], padding=True, truncation=True)

In [103]:
encoded_dataset = dataset.map(tokenize, batch_size=None, batched=True)

Map:   0%|          | 0/3897 [00:00<?, ? examples/s]

Map:   0%|          | 0/1114 [00:00<?, ? examples/s]

Map:   0%|          | 0/557 [00:00<?, ? examples/s]

In [104]:
from transformers import AutoModelForSequenceClassification, AutoConfig
import torch

label2id = {"ham": 0, "spam": 1}
id2label = {0:"ham", 1:"spam"}

model_ckpt = "distilbert-base-uncased"
# model_ckpt = "google/mobilebert-uncased"
# model_ckpt = "huawei-noah/TinyBERT_General_4L_312D"


num_labels = len(label2id)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = AutoConfig.from_pretrained(model_ckpt, label2id=label2id, id2label=id2label)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, config=config).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [105]:
from transformers import TrainingArguments

batch_size = 32
training_dir =  r"C:\Temp\bert_base_train_dir"

training_args = TrainingArguments(
                                  output_dir=training_dir,
                                  num_train_epochs = 2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = 'epoch'
)

c:\Users\Pc\anaconda3\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [106]:
# use sklearn to build compute metrics
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}

In [107]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=distilbert_tokenizer
)

In [108]:
trainer.train()

  0%|          | 0/244 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.02609287016093731, 'eval_accuracy': 0.9964093357271095, 'eval_f1': 0.9963887915257832, 'eval_runtime': 31.3627, 'eval_samples_per_second': 17.76, 'eval_steps_per_second': 0.574, 'epoch': 1.0}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.03156541287899017, 'eval_accuracy': 0.9910233393177738, 'eval_f1': 0.9910484025805975, 'eval_runtime': 30.1351, 'eval_samples_per_second': 18.483, 'eval_steps_per_second': 0.597, 'epoch': 2.0}
{'train_runtime': 2527.3573, 'train_samples_per_second': 3.084, 'train_steps_per_second': 0.097, 'train_loss': 0.07468587844098201, 'epoch': 2.0}


TrainOutput(global_step=244, training_loss=0.07468587844098201, metrics={'train_runtime': 2527.3573, 'train_samples_per_second': 3.084, 'train_steps_per_second': 0.097, 'total_flos': 479928350429136.0, 'train_loss': 0.07468587844098201, 'epoch': 2.0})

In [109]:
preds_output = trainer.predict(encoded_dataset['test'])

  0%|          | 0/35 [00:00<?, ?it/s]

In [110]:
preds_output.metrics

{'test_loss': 0.029494360089302063,
 'test_accuracy': 0.9910233393177738,
 'test_f1': 0.9909715565985184,
 'test_runtime': 130.3888,
 'test_samples_per_second': 8.544,
 'test_steps_per_second': 0.268}

In [111]:
y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = encoded_dataset['test'][:]['label']

In [112]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=list(label2id)))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99       965
        spam       0.98      0.95      0.97       149

    accuracy                           0.99      1114
   macro avg       0.99      0.97      0.98      1114
weighted avg       0.99      0.99      0.99      1114

